In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

import requests

In [7]:
url_inflation_db = "https://github.com/matublaq/DataBases/raw/main/Inflation.db"
local_path = "Inflation.db"

response = requests.get(url_inflation_db)

if(response.status_code == 200):
    with open(local_path, 'wb') as file:
        file.write(response.content)
    print(f"Inflation database. Downloaded successfully. Status code {response.status_code}")
else:
    print(f"Inflation database. Download failed. Status code {response.status_code}")

################################################################################################################

url_stockmarket_db = "https://github.com/matublaq/DataBases/raw/main/StockMarket.db"
local_path = "StockMarket.db"

response = requests.get(url_stockmarket_db)

if(response.status_code == 200):
    with open(local_path, 'wb') as file:
        file.write(response.content)
    print(f"Stock market database. Downloaded successfully. Status code {response.status_code}")
else:
    print(f"Stock market database. Download failed. Status code {response.status_code}")

Inflation database. Downloaded successfully. Status code 200
Stock market database. Downloaded successfully. Status code 200


In [10]:
conn = sqlite3.connect('Inflation.db')
cursor = conn.cursor()

cursor.execute("SELECT name FROM Countries")
companies = cursor.fetchall()

conn.close()

print(companies)

[('Algeria',), ('Angola',), ('Antigua and Barbuda',), ('Argentina',), ('Australia',), ('Austria',), ('Bahamas, The',), ('Bahrain',), ('Bangladesh',), ('Barbados',), ('Belarus',), ('Belgium',), ('Belize',), ('Benin',), ('Bolivia',), ('Botswana',), ('Brazil',), ('Brunei Darussalam',), ('Bulgaria',), ('Burkina Faso',), ('Burundi',), ('Cabo Verde',), ('Cameroon',), ('Canada',), ('Central African Republic',), ('Chad',), ('Chile',), ('China',), ('Colombia',), ('Comoros',), ('Congo, Dem. Rep.',), ('Congo, Rep.',), ('Costa Rica',), ("Côte d'Ivoire",), ('Cyprus',), ('Czech Republic',), ('Denmark',), ('Djibouti',), ('Dominica',), ('Dominican Republic',), ('Ecuador',), ('Egypt, Arab Rep.',), ('El Salvador',), ('Equatorial Guinea',), ('Ethiopia',), ('Fiji',), ('Finland',), ('France',), ('Gabon',), ('Gambia, The',), ('Germany',), ('Ghana',), ('Greece',), ('Grenada',), ('Guatemala',), ('Guinea',), ('Guinea-Bissau',), ('Guyana',), ('Haiti',), ('Honduras',), ('Hungary',), ('Iceland',), ('India',), ('I